![](images/EscUpmPolit_p.gif "UPM")

# Course Notes for Learning Intelligent Systems

Department of Telematic Engineering Systems, Universidad Politécnica de Madrid, © Carlos Á. Iglesias

## [Introduction to Machine Learning V](2_6_0_Intro_RL.ipynb)

# Introduction
The purpose of this practice is to understand better Reinforcement Learning (RL) and, in particular, Q-Learning.

We are going to use [Gymnasium](https://gymnasium.farama.org/). Gymnasium is toolkit for developing and comparing RL algorithms. It is a fork of [Open AI Gym](https://www.gymlibrary.dev/). Take a loot at ther [website](https://github.com/Farama-Foundation/Gymnasium).

It implements [algorithm imitation](http://gym.openai.com/envs/#algorithmic), [classic control problems](https://gymnasium.farama.org/environments/classic_control/), [Atari games](https://gymnasium.farama.org/environments/atari/), [Box2D continuous control](https://gymnasium.farama.org/environments/box2d/), [robotics with MuJoCo, Multi-Joint dynamics with Contact](https://gymnasium.farama.org/environments/mujoco/), [simple text based environments](https://gymnasium.farama.org/environments/toy_text/), and [other problems](https://gymnasium.farama.org/environments/third_party_environments/).

This notebook is based on [Diving deeper into Reinforcement Learning with Q-Learning](https://medium.freecodecamp.org/diving-deeper-into-reinforcement-learning-with-q-learning-c18d0db58efe) and [Introduction to Q-Learning](https://huggingface.co/deep-rl-course/unit2/hands-on?fw=pt).

First of all, install the Gymnasium library, which is a fork of the OpenAI Gym  library:

```console
foo@bar:~$ conda install gymnasium
```

If you get an error 'No module named 'Box2D', install 'pybox2d'.


# Getting started with Gymnasium

First of all, read the [introduction](https://gymnasium.farama.org/content/basic_usage/) of Gymnasium.

## Environments
OpenGym provides a number of problems called *environments*. 

Try 'LunarLander-v2' (or 'CartPole-v1', 'MountainCar-v0', etc.).

In [1]:
from warnings import filterwarnings
filterwarnings(action='ignore', category=DeprecationWarning)


import gymnasium as gym
#env = gym.make("LunarLander-v2", render_mode="human")
#env = gym.make("CartPole-v1", render_mode="human")
env = gym.make("MountainCar-v0", render_mode="human")
observation, info = env.reset()

for _ in range(100):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward, terminated, truncated, info = env.step(action)

    if terminated or truncated:
        observation, info = env.reset()

env.close()

This will launch an external window with the game. If you cannot close that window, just execute in a code cell:

```python
env.close()
```

The full list of available environments can be found printing the environment registry as follows.

In [2]:
gym.envs.registry.keys()

dict_keys(['CartPole-v0', 'CartPole-v1', 'MountainCar-v0', 'MountainCarContinuous-v0', 'Pendulum-v1', 'Acrobot-v1', 'CartPoleJax-v0', 'CartPoleJax-v1', 'PendulumJax-v0', 'LunarLander-v2', 'LunarLanderContinuous-v2', 'BipedalWalker-v3', 'BipedalWalkerHardcore-v3', 'CarRacing-v2', 'Blackjack-v1', 'FrozenLake-v1', 'FrozenLake8x8-v1', 'CliffWalking-v0', 'Taxi-v3', 'Reacher-v2', 'Reacher-v4', 'Pusher-v2', 'Pusher-v4', 'InvertedPendulum-v2', 'InvertedPendulum-v4', 'InvertedDoublePendulum-v2', 'InvertedDoublePendulum-v4', 'HalfCheetah-v2', 'HalfCheetah-v3', 'HalfCheetah-v4', 'Hopper-v2', 'Hopper-v3', 'Hopper-v4', 'Swimmer-v2', 'Swimmer-v3', 'Swimmer-v4', 'Walker2d-v2', 'Walker2d-v3', 'Walker2d-v4', 'Ant-v2', 'Ant-v3', 'Ant-v4', 'Humanoid-v2', 'Humanoid-v3', 'Humanoid-v4', 'HumanoidStandup-v2', 'HumanoidStandup-v4', 'GymV22Environment-v0', 'GymV26Environment-v0'])

You can check the environment specification with spec.

In [3]:
print(env.spec)

EnvSpec(id='MountainCar-v0', entry_point='gymnasium.envs.classic_control.mountain_car:MountainCarEnv', reward_threshold=-110.0, nondeterministic=False, max_episode_steps=200, order_enforce=True, autoreset=False, disable_env_checker=False, apply_api_compatibility=False, kwargs={'render_mode': 'human'}, namespace=None, name='MountainCar', version=0)


The environment’s **step** function returns five values. These are:

* **observation (object):** an environment-specific object representing your observation of the environment. For example, pixel data from a camera, joint angles and joint velocities of a robot, or the board state in a board game.
* **reward (float):** amount of reward achieved by the previous action. The scale varies between environments, but the goal is always to increase your total reward.
* **terminated (boolean):** whether the agent reaches the terminal state, which can be positive (e.g., reaching the goal state) or negative (e.g., you lost your last life). If true, the user needs to call *reset()*.
* **truncated (boolean):** when the truncated condition is satisfied (e.g., timelimit or mechanical problem in a robot). It can be used  to end the episode prematurely  before a terminal state is reached . If true, the user needs to call *reset()*.
* **info (dict):** diagnostic information useful for debugging. It can sometimes be useful for learning (for example, it might contain the raw probabilities behind the environment’s last state change). However, official evaluations of your agent are not allowed to use this for learning.

The typical agent loop consists in first calling the method *reset* which provides an initial observation. Then the agent executes an action, and receives the reward, the new observation, and if the episode has finished (terminated or truncated are true). 

For example, analyze this sample of agent loop for 100 ms. The details of the previous variables for this game as described [here](https://gymnasium.farama.org/environments/classic_control/cart_pole/) are:
* **observation**: Cart Position, Cart Velocity, Pole Angle, Pole Velocity.
* **action**: 0	(Push cart to the left), 1	(Push cart to the right).
* **reward**: 1  for every step taken, including the termination step.

In [4]:
import gymnasium as gym

env = gym.make('CartPole-v1', render_mode='human')
for i_episode in range(10):
    
    
    observation = env.reset()
    for t in range(10):
        env.render()
        print(observation)
        action = env.action_space.sample()
        print("Action ", action)
        observation, reward,  terminated, truncated, info = env.step(action)
        print("Observation ", observation, ", reward ", reward, ", terminated ", terminated,
              ", truncated", truncated, ", info " , info)
        if terminated or truncated:
            print("Episode finished after {} timesteps".format(t+1))
            break

(array([0.02466699, 0.02517318, 0.04970684, 0.01225195], dtype=float32), {})
Action  0
Observation  [ 0.02517045 -0.17062509  0.04995188  0.3201944 ] , reward  1.0 , terminated  False , truncated False , info  {}
[ 0.02517045 -0.17062509  0.04995188  0.3201944 ]
Action  0
Observation  [ 0.02175795 -0.36642152  0.05635576  0.62820244] , reward  1.0 , terminated  False , truncated False , info  {}
[ 0.02175795 -0.36642152  0.05635576  0.62820244]
Action  0
Observation  [ 0.01442952 -0.56228286  0.06891982  0.9380878 ] , reward  1.0 , terminated  False , truncated False , info  {}
[ 0.01442952 -0.56228286  0.06891982  0.9380878 ]
Action  1
Observation  [ 0.00318386 -0.3681544   0.08768157  0.66783285] , reward  1.0 , terminated  False , truncated False , info  {}
[ 0.00318386 -0.3681544   0.08768157  0.66783285]
Action  1
Observation  [-0.00417923 -0.17435414  0.10103822  0.4039946 ] , reward  1.0 , terminated  False , truncated False , info  {}
[-0.00417923 -0.17435414  0.10103822  0.403

[ 0.0185852  -0.00988272 -0.0521947  -0.08780695]
Action  0
Observation  [ 0.01838755 -0.20421918 -0.05395084  0.18796247] , reward  1.0 , terminated  False , truncated False , info  {}
[ 0.01838755 -0.20421918 -0.05395084  0.18796247]
Action  0
Observation  [ 0.01430316 -0.3985294  -0.05019159  0.46314988] , reward  1.0 , terminated  False , truncated False , info  {}
[ 0.01430316 -0.3985294  -0.05019159  0.46314988]
Action  1
Observation  [ 0.00633258 -0.2027354  -0.04092859  0.1550786 ] , reward  1.0 , terminated  False , truncated False , info  {}
[ 0.00633258 -0.2027354  -0.04092859  0.1550786 ]
Action  0
Observation  [ 0.00227787 -0.39724815 -0.03782702  0.43457374] , reward  1.0 , terminated  False , truncated False , info  {}
(array([-0.01675624,  0.00657571, -0.01191216, -0.03199182], dtype=float32), {})
Action  1
Observation  [-0.01662473  0.20186645 -0.012552   -0.32840922] , reward  1.0 , terminated  False , truncated False , info  {}
[-0.01662473  0.20186645 -0.012552   -0

Observation  [ 0.03367834  0.22120321 -0.04192962 -0.2881138 ] , reward  1.0 , terminated  False , truncated False , info  {}
[ 0.03367834  0.22120321 -0.04192962 -0.2881138 ]
Action  0
Observation  [ 0.0381024   0.0267035  -0.0476919  -0.00894437] , reward  1.0 , terminated  False , truncated False , info  {}
[ 0.0381024   0.0267035  -0.0476919  -0.00894437]
Action  0
Observation  [ 0.03863648 -0.16770318 -0.04787079  0.268318  ] , reward  1.0 , terminated  False , truncated False , info  {}
[ 0.03863648 -0.16770318 -0.04787079  0.268318  ]
Action  1
Observation  [ 0.03528241  0.02806809 -0.04250443 -0.03907115] , reward  1.0 , terminated  False , truncated False , info  {}
[ 0.03528241  0.02806809 -0.04250443 -0.03907115]
Action  1
Observation  [ 0.03584377  0.22377296 -0.04328585 -0.34485587] , reward  1.0 , terminated  False , truncated False , info  {}
[ 0.03584377  0.22377296 -0.04328585 -0.34485587]
Action  0
Observation  [ 0.04031923  0.02929264 -0.05018297 -0.06613071] , rewar

# The Frozen Lake scenario
We are going to play to the [Frozen Lake](https://gymnasium.farama.org/environments/toy_text/frozen_lake/) game.

The problem is a grid where you should go from the 'start' (S) position to the 'goal position (G) (the pizza!). You can only walk through the 'frozen tiles' (F). Unfortunately, you can fall in a  'hole' (H).
![](images/frozenlake-problem.png "Frozen lake problem")

The episode ends when you reach the goal or fall in a hole. You receive a reward of 1 if you reach the goal, and zero otherwise. The possible actions are going left, right, up or down. However, the ice is slippery, so you won't always move in the direction you intend.

![](images/frozenlake-world.png "Frozen lake world")


Here you can see several episodes. A full recording is available at  [Frozen World](http://gym.openai.com/envs/FrozenLake-v0/).

![](images/recording.gif "Example running")


# Q-Learning with the Frozen Lake scenario
We are now going to apply Q-Learning for the Frozen Lake scenario. This part of the notebook is taken from [here](https://github.com/simoninithomas/Deep_reinforcement_learning_Course/blob/master/Q%20learning/Q%20Learning%20with%20FrozenLake.ipynb). You can get more details about this scenario [here](https://gymnasium.farama.org/environments/toy_text/frozen_lake/).

First we create the environment and a Q-table inizializated with zeros to store the value of each action in a given state. 

In [5]:
import numpy as np
import gymnasium as gym
import random

env = gym.make("FrozenLake-v1", desc=None, map_name="4x4", is_slippery=False) #no render so training is faster
#env = gym.make("FrozenLake-v1", desc=None, map_name="4x4", is_slippery=False, render_mode='human')
#env = gym.make("FrozenLake-v1", desc=None, map_name="4x4", is_slippery=False, render_mode='rgb_array')


action_space = env.action_space.n
state_space = env.observation_space.n

print("There are ", state_space, " possible states")
print("There are ", action_space, " possible actions")

# Step 1. Initialize QTable
qtable = np.zeros((state_space, action_space))
print("QTable")
print(qtable)

There are  16  possible states
There are  4  possible actions
QTable
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


Now we define the hyperparameters.

In [6]:
# Q-Learning hyperparameters
total_episodes = 10000        # Total episodes
learning_rate = 0.8           # Learning rate
max_steps = 99                # Max steps per episode
gamma = 0.95                  # Discounting rate

# Exploration hyperparameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.01             # Exponential decay rate for exploration prob

And now we implement the Q-Learning algorithm.

![](images/qlearning-algo.png "Q-Learning algorithm")

In [7]:
state, info = env.reset() #reset returns observation, info
step = 0
done = False
total_rewards = 0

In [8]:
# Traing to learn Q-Table
from tqdm.notebook import tqdm
# List of rewards
rewards = []

# Step 2 For life or until learning is stopped
for episode in tqdm(range(total_episodes)):
    # Reset the environment
    state, info = env.reset() #reset returns observation, info
    step = 0
    done = False
    total_rewards = 0
    
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0, 1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state][:])

        # Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()

        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, terminated, truncated, info = env.step(action)

        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state][:] : all the actions we can take from new state
        qtable[state][action] = qtable[state][action] + learning_rate * (reward + gamma * np.max(qtable[new_state]) - qtable[state][action])
        
        total_rewards += reward
        
        # Our new state is state
        state = new_state
        
        done = terminated or truncated
        # If done (if we're dead) : finish episode
        if done == True: 
            break
        
    episode += 1
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode) 
    rewards.append(total_rewards)

print ("Score over time: " +  str(sum(rewards)/total_episodes))
print(qtable)

  0%|          | 0/10000 [00:00<?, ?it/s]

Score over time: 0.9765
[[0.73509189 0.77378094 0.6983373  0.73509189]
 [0.73509189 0.         0.         0.        ]
 [0.5586684  0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.77378094 0.81450625 0.         0.73509189]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.81450625 0.         0.857375   0.77378094]
 [0.81450625 0.9025     0.81450625 0.        ]
 [0.857375   0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.9025     0.95       0.857375  ]
 [0.9025     0.95       1.         0.81450625]
 [0.         0.         0.         0.        ]]


Finally, we use the learnt Q-table for playing the Frozen World game.

In [9]:
env.reset()

for episode in range(5):
    state, info = env.reset()
    step = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode)
    print("****************************************************")

    for step in range(max_steps):
        env.render() # render according to rend_mode specified in gym.make
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state][:])
        print("Action ", action)
        
        new_state, reward, terminated, truncated, info = env.step(action)
        
        if done:
            break
        state = new_state
env.close()

****************************************************
EPISODE  0
****************************************************
Action  1
Action  1
Action  2
Action  1
Action  2
Action  2
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Action  0
Act

/home/cif/anaconda3/lib/python3.10/site-packages/gymnasium/envs/toy_text/frozen_lake.py:328: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("FrozenLake-v1", render_mode="rgb_array")
  gym.logger.warn(


## References
* [Gymnasium documentation](https://gymnasium.farama.org/).
* [Diving deeper into Reinforcement Learning with Q-Learning, Thomas Simonini](https://medium.freecodecamp.org/diving-deeper-into-reinforcement-learning-with-q-learning-c18d0db58efe).
* Illustrations by [Thomas Simonini](https://github.com/simoninithomas/Deep_reinforcement_learning_Course) and [Sung Kim](https://www.youtube.com/watch?v=xgoO54qN4lY).
* [Frozen Lake solution with TensorFlow](https://analyticsindiamag.com/openai-gym-frozen-lake-beginners-guide-reinforcement-learning/)
* [Deep Q-Learning for Doom](https://medium.freecodecamp.org/an-introduction-to-deep-q-learning-lets-play-doom-54d02d8017d8)
* [Intro OpenAI Gym with Random Search and the Cart Pole scenario](http://www.pinchofintelligence.com/getting-started-openai-gym/)
* [Q-Learning for the Taxi scenario](https://www.oreilly.com/learning/introduction-to-reinforcement-learning-and-openai-gym)

## Licence

The notebook is freely licensed under under the [Creative Commons Attribution Share-Alike license](https://creativecommons.org/licenses/by/2.0/).  

© Carlos Á. Iglesias, Universidad Politécnica de Madrid.